# テキスト操作の比較
## ライブラリ準備

In [8]:
import logging
import time
import json
import os

from dotenv import load_dotenv
import requests

from typing import List, Dict

import cohere
import anthropic
import openai
from openai import OpenAI

In [9]:
load_dotenv()

CLAUDE_API_KEY = os.getenv("CLAUDE_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## 各サービスへのクライアントの定義

In [10]:
qa_query = lambda text: f"""以下の依頼について、
ビジネス上の顧客への回答として敬語表現を適切に回答してください。
   
依頼テキスト:
{text}

回答日本語:"""

class AIModelClient:

    def _query(self, func, text: str, model: str = None) -> Dict:
        start_time = time.time()
        try:
            translation, model = func(text, model)
            elapsed_time = time.time() - start_time
            
            return {
                "result": translation,
                "time": elapsed_time,
                "model": model,
                "success": True,
                "error": None
            }
        except Exception as e:
            return {
                "result": None,
                "time": time.time() - start_time,
                "model": model,
                "success": False,
                "error": str(e)
            }

    def answer(self, text: str, model: str = None) -> Dict:
        return self._query(self._concrete_answer, text, model)

class ClaudeClient(AIModelClient):
    def __init__(self, api_key):
        self.client = anthropic.Anthropic(api_key=api_key)

    def _concrete_answer(self, text: str, model: str) -> Dict:
        if model is None:
            model = "claude-sonnet-4-5"
        message = self.client.messages.create(
            model=model,
            max_tokens=4096,
            messages=[
                {
                    "role": "user",
                    "content": qa_query(text)
                }
            ]
        )        
        return message.content[0].text, model

claude_client = ClaudeClient(CLAUDE_API_KEY)

In [11]:
class CohereClient(AIModelClient):
    def __init__(self, api_key):
        self.client = cohere.ClientV2(api_key=api_key)

    # https://docs.cohere.com/docs/command-a-translate
    def _concrete_answer(self, text: str, model: str) -> Dict:
        if model is None:
            model = "command-a-translate-08-2025"
        response = self.client.chat(
            model=model,
            messages=[
                {
                    "role": "user",
                    "content": qa_query(text)
                }
            ],
        )
        return response.message.content[0].text, model
    
cohere_client = CohereClient(COHERE_API_KEY)

In [12]:
class OpenAIClient(AIModelClient):
    def __init__(self, api_key):
        self.client = OpenAI(api_key=api_key)

    def _concrete_answer(self, text: str, model: str) -> Dict:
        if model is None:
            model  = "gpt-5-mini"

        response = self.client.chat.completions.create(
            model=model,
            messages=[
                {
                    "role": "system",
                    "content": "あなたは優秀な翻訳者です。英語を自然で流暢な日本語に翻訳してください。"
                },
                {
                    "role": "user",
                    "content": qa_query(text)
                }
            ],
            temperature=0.3
        )        
        return response.choices[0].message.content, model

openai_client = OpenAIClient(OPENAI_API_KEY)

In [24]:
def display_answer(response):
    print(f'##### model: {response["model"]}, time: {response["time"]:.2f}sec #####')
    print(response['result'])
    print('')

text = "おいしい七草粥の作り方教えて。"
display_answer(claude_client.answer(text))
display_answer(cohere_client.answer(text))

##### model: claude-sonnet-4-5, time: 9.77sec #####
いつもお世話になっております。

七草粥の作り方についてご案内申し上げます。

**材料（2人分）**
- お米：1/2カップ
- 水：約5カップ
- 春の七草（セリ、ナズナ、ゴギョウ、ハコベラ、ホトケノザ、スズナ、スズシロ）
- 塩：小さじ1/2程度

**作り方**

1. お米は洗って30分ほど水に浸し、ザルに上げて水気を切ります

2. 鍋にお米と水を入れ、強火にかけます。沸騰したら弱火にし、蓋を少しずらして40分ほど炊きます

3. 七草は塩少々を加えた熱湯でサッと茹で、冷水にとって水気を絞り、細かく刻みます

4. お粥が炊き上がりましたら、刻んだ七草を加えて軽く混ぜ、塩で味を調えます

5. 2〜3分蒸らしてから器に盛り付けてお召し上がりください

**美味しく作るポイント**
- 七草は茹ですぎず、色鮮やかに仕上げることで風味が増します
- お好みでお餅を加えても美味しくお召し上がりいただけます

ご参考になれば幸いでございます。
他にご不明な点がございましたら、お気軽にお申し付けくださいませ。

##### model: command-a-translate-08-2025, time: 6.67sec #####
七草粥の美味しい作り方についてご質問いただき、ありがとうございます。

七草粥は、日本の伝統的な行事食で、1月7日の「七草がゆ」の日に食べる習慣があります。七草（セリ、ナズナ、ゴギョウ、ハコベラ、ホトケノザ、スズナ、スズシロ）を刻んでお粥に入れることで、冬の寒さで弱った胃腸を整え、健康を願う意味があります。

以下に、美味しい七草粥の作り方をご紹介いたします。

1. **材料の準備**:
   - 七草（セリ、ナズナ、ゴギョウ、ハコベラ、ホトケノザ、スズナ、スズシロ） 適量
   - 米 1合
   - 水 適量（米の2倍程度）
   - 塩 少々

2. **七草の下準備**:
   - 七草をよく洗い、水気を切って細かく刻みます。

3. **お粥の調理**:
   - 鍋に米と水を入れ、中火で炊きます。
   - 沸騰したら弱火にし、蓋をして約15分間炊きます。
   - 米が柔らかくなったら、刻んだ七草を加え、さらに5分ほ

In [25]:
text = "商品が壊れていたので返品したい。"
display_answer(claude_client.answer(text))
display_answer(cohere_client.answer(text))

##### model: claude-sonnet-4-5, time: 5.63sec #####
お問い合わせいただき、誠にありがとうございます。

この度は、お届けいたしました商品が破損しており、大変申し訳ございません。
お客様にはご迷惑をおかけいたしましたこと、心よりお詫び申し上げます。

つきましては、返品のお手続きをさせていただきたく存じます。
恐れ入りますが、以下の情報をお知らせいただけますでしょうか。

・ご注文番号
・商品名
・破損箇所の詳細（可能であれば写真も添付いただけますと幸いです）

確認でき次第、速やかに返品・交換または返金の手続きを進めさせていただきます。

ご不便をおかけして誠に申し訳ございませんが、何卒よろしくお願い申し上げます。

##### model: command-a-translate-08-2025, time: 2.66sec #####
承知いたしました。商品が壊れていたとのこと、大変申し訳ございません。

返品をご希望とのことですが、以下の手順で対応させていただきます。

1. まず、お手数ですが、商品の状態を確認させていただくため、写真をご送付いただけますでしょうか。
2. 写真を確認後、返品手続きの詳細をご案内いたします。
3. 返品商品が到着次第、交換品または返金の手配をいたします。

お手数をおかけいたしますが、何卒よろしくお願い申し上げます。

ご不明な点がございましたら、お気軽にお問い合わせください。



In [26]:
text = "かのっwkらぬsっっlkなえあおわえうrl"
display_answer(claude_client.answer(text))
display_answer(cohere_client.answer(text))

##### model: claude-sonnet-4-5, time: 2.87sec #####
申し訳ございませんが、お送りいただいたメッセージの内容を正しく読み取ることができませんでした。

お手数をおかけいたしますが、もう一度ご用件をお書きいただけますでしょうか。

何卒よろしくお願い申し上げます。

##### model: command-a-translate-08-2025, time: 0.99sec #####
申し訳ございませんが、ご依頼の内容が不明確でございます。恐れ入りますが、再度ご確認の上、ご依頼いただけますと幸いでございます。何卒よろしくお願い申し上げます。



※この Notebook は、API の使い方を試す目的のものです。 AI の出力は不正確な場合があります。※

In [17]:
print(openai_client.answer(text))

{'result': None, 'time': 3.5559139251708984, 'model': None, 'success': False, 'error': "Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}"}
